# NOAA GHCN Metadata Exploration

This notebook explores the metadata files from the NOAA Global Historical Climatology Network (GHCN) dataset stored in S3.

**Data Source:** `s3://noaa-ghcn-pds/`

**Metadata Files:**
- `ghcnd-stations.txt` - Weather station locations and info
- `ghcnd-countries.txt` - Country code mappings
- `ghcnd-states.txt` - US state/Canadian province codes
- `ghcnd-inventory.txt` - Data availability per station

In [2]:
# Install required packages if needed
! pip install pandas s3fs fsspec

zsh:1: command not found: pip


In [3]:
import pandas as pd
import s3fs
from io import StringIO

# S3 filesystem (no credentials needed for public bucket)
S3_BUCKET = 's3://noaa-ghcn-pds'
fs = s3fs.S3FileSystem(anon=True)

## 1. Countries Metadata

In [4]:
# Read countries file (format: 2-char code + space + country name)
with fs.open('noaa-ghcn-pds/ghcnd-countries.txt', 'r') as f:
    countries_raw = f.read()

countries = pd.DataFrame([
    {'country_code': line[:2], 'country_name': line[3:].strip()}
    for line in countries_raw.strip().split('\n')
])

print(f"Total countries: {len(countries)}")
countries.head(10)

Total countries: 219


,country_code,country_name
0,AC,Antigua and Barbuda
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,Algeria
4,AJ,Azerbaijan
5,AL,Albania
6,AM,Armenia
7,AO,Angola
8,AQ,American Samoa [United States]
9,AR,Argentina


In [5]:
# View all countries
countries

,country_code,country_name
0,AC,Antigua and Barbuda
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,Algeria
4,AJ,Azerbaijan
...,...,...
214,WI,Western Sahara
215,WQ,Wake Island [United States]
216,WZ,Swaziland
217,ZA,Zambia


## 2. States/Provinces Metadata

In [6]:
# Read states file (format: 2-char code + space + state name)
with fs.open('noaa-ghcn-pds/ghcnd-states.txt', 'r') as f:
    states_raw = f.read()

states = pd.DataFrame([
    {'state_code': line[:2], 'state_name': line[3:].strip()}
    for line in states_raw.strip().split('\n')
])

print(f"Total states/provinces: {len(states)}")
states

Total states/provinces: 74


,state_code,state_name
0,AB,ALBERTA
1,AK,ALASKA
2,AL,ALABAMA
3,AR,ARKANSAS
4,AS,AMERICAN SAMOA
...,...,...
69,WA,WASHINGTON
70,WI,WISCONSIN
71,WV,WEST VIRGINIA
72,WY,WYOMING


## 3. Stations Metadata

Fixed-width format:
- Columns 1-11: Station ID
- Columns 13-20: Latitude
- Columns 22-30: Longitude  
- Columns 32-37: Elevation (meters)
- Columns 39-40: State code
- Columns 42-71: Station name
- Columns 73-75: GSN flag
- Columns 77-79: HCN/CRN flag
- Columns 81-85: WMO ID

In [7]:
# Read stations file (fixed-width format)
with fs.open('noaa-ghcn-pds/ghcnd-stations.txt', 'r') as f:
    stations_raw = f.read()

stations = pd.read_fwf(
    StringIO(stations_raw),
    colspecs=[
        (0, 11),    # ID
        (12, 20),   # LATITUDE
        (21, 30),   # LONGITUDE
        (31, 37),   # ELEVATION
        (38, 40),   # STATE
        (41, 71),   # NAME
        (72, 75),   # GSN_FLAG
        (76, 79),   # HCN_CRN_FLAG
        (80, 85)    # WMO_ID
    ],
    names=['station_id', 'latitude', 'longitude', 'elevation', 'state', 'name', 'gsn_flag', 'hcn_crn_flag', 'wmo_id']
)

print(f"Total stations: {len(stations):,}")
stations.head(10)

Total stations: 129,658


,station_id,latitude,longitude,elevation,state,name,gsn_flag,hcn_crn_flag,wmo_id
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217.0
5,AEM00041218,24.2620,55.6090,264.9,NaN,AL AIN INTL,NaN,NaN,41218.0
6,AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
7,AFM00040938,34.2100,62.2280,977.2,NaN,HERAT,NaN,NaN,40938.0
8,AFM00040948,34.5660,69.2120,1791.3,NaN,KABUL INTL,NaN,NaN,40948.0
9,AFM00040990,31.5000,65.8500,1010.0,NaN,KANDAHAR AIRPORT,NaN,NaN,40990.0


In [8]:
# Station statistics
print("Station Statistics:")
print(f"  Total stations: {len(stations):,}")
print(f"  Elevation range: {stations['elevation'].min():.1f}m to {stations['elevation'].max():.1f}m")
print(f"  Latitude range: {stations['latitude'].min():.2f} to {stations['latitude'].max():.2f}")
print(f"  Longitude range: {stations['longitude'].min():.2f} to {stations['longitude'].max():.2f}")

Station Statistics:
  Total stations: 129,658
  Elevation range: -999.9m to 5033.0m
  Latitude range: -90.00 to 83.65
  Longitude range: -179.99 to 179.74


In [9]:
# Stations by country (first 2 chars of station_id = country code)
stations['country_code'] = stations['station_id'].str[:2]
stations_by_country = stations.groupby('country_code').size().reset_index(name='station_count')
stations_by_country = stations_by_country.merge(countries, on='country_code', how='left')
stations_by_country = stations_by_country.sort_values('station_count', ascending=False)

print("Top 20 countries by number of stations:")
stations_by_country.head(20)

Top 20 countries by number of stations:


,country_code,station_count,country_name
205,US,75847,United States
10,AS,17088,Australia
31,CA,9269,Canada
27,BR,5989,Brazil
137,MX,5249,Mexico
89,IN,3807,India
185,SW,1721,Sweden
175,SF,1166,South Africa
77,GM,1123,Germany
170,RS,1123,Russia


In [10]:
# US stations by state
us_stations = stations[stations['country_code'] == 'US'].copy()
us_by_state = us_stations.groupby('state').size().reset_index(name='station_count')
us_by_state = us_by_state.merge(states, left_on='state', right_on='state_code', how='left')
us_by_state = us_by_state.sort_values('station_count', ascending=False)

print(f"Total US stations: {len(us_stations):,}")
us_by_state.head(20)

Total US stations: 75,847


,state,station_count,state_code,state_name
43,TX,6472,TX,TEXAS
5,CO,4784,CO,COLORADO
4,CA,3166,CA,CALIFORNIA
27,NC,2747,NC,NORTH CAROLINA
23,MN,2675,MN,MINNESOTA
29,NE,2436,NE,NEBRASKA
16,KS,2401,KS,KANSAS
32,NM,2295,NM,NEW MEXICO
9,FL,2244,FL,FLORIDA
14,IL,2234,IL,ILLINOIS


## 4. Inventory Metadata

Shows what data elements are available for each station and the date range.

Fixed-width format:
- Columns 1-11: Station ID
- Columns 13-20: Latitude
- Columns 22-30: Longitude
- Columns 32-35: Element (TMAX, TMIN, PRCP, etc.)
- Columns 37-40: First year
- Columns 42-45: Last year

In [11]:
# Read inventory file (fixed-width format)
with fs.open('noaa-ghcn-pds/ghcnd-inventory.txt', 'r') as f:
    inventory_raw = f.read()

inventory = pd.read_fwf(
    StringIO(inventory_raw),
    colspecs=[
        (0, 11),    # ID
        (12, 20),   # LATITUDE
        (21, 30),   # LONGITUDE
        (31, 35),   # ELEMENT
        (36, 40),   # FIRST_YEAR
        (41, 45)    # LAST_YEAR
    ],
    names=['station_id', 'latitude', 'longitude', 'element', 'first_year', 'last_year']
)

print(f"Total inventory records: {len(inventory):,}")
inventory.head(10)

Total inventory records: 767,689


,station_id,latitude,longitude,element,first_year,last_year
0,ACW00011604,17.1167,-61.7833,TMAX,1949,1949
1,ACW00011604,17.1167,-61.7833,TMIN,1949,1949
2,ACW00011604,17.1167,-61.7833,PRCP,1949,1949
3,ACW00011604,17.1167,-61.7833,SNOW,1949,1949
4,ACW00011604,17.1167,-61.7833,SNWD,1949,1949
5,ACW00011604,17.1167,-61.7833,PGTM,1949,1949
6,ACW00011604,17.1167,-61.7833,WDFG,1949,1949
7,ACW00011604,17.1167,-61.7833,WSFG,1949,1949
8,ACW00011604,17.1167,-61.7833,WT03,1949,1949
9,ACW00011604,17.1167,-61.7833,WT08,1949,1949


In [12]:
# Available data elements
elements = inventory.groupby('element').agg(
    station_count=('station_id', 'nunique'),
    earliest_year=('first_year', 'min'),
    latest_year=('last_year', 'max')
).reset_index().sort_values('station_count', ascending=False)

print("Data elements available:")
print("\nCommon elements:")
print("  TMAX = Maximum temperature")
print("  TMIN = Minimum temperature")
print("  PRCP = Precipitation")
print("  SNOW = Snowfall")
print("  SNWD = Snow depth")
print()
elements.head(20)

Data elements available:

Common elements:
  TMAX = Maximum temperature
  TMIN = Minimum temperature
  PRCP = Precipitation
  SNOW = Snowfall
  SNWD = Snow depth



,element,station_count,earliest_year,latest_year
32,PRCP,127478,1781,2026
65,SNOW,80950,1840,2026
24,MDPR,72524,1832,2026
66,SNWD,67217,1850,2026
11,DAPR,65271,1832,2026
99,TMAX,40439,1763,2026
100,TMIN,40336,1763,2026
110,WESD,26590,1952,2026
111,WESF,26044,1998,2026
118,WT01,17114,1843,2025


In [13]:
# Data coverage over time
inventory['years_of_data'] = inventory['last_year'] - inventory['first_year'] + 1

print("Data coverage statistics:")
print(f"  Earliest data: {inventory['first_year'].min()}")
print(f"  Latest data: {inventory['last_year'].max()}")
print(f"  Average years of data per station-element: {inventory['years_of_data'].mean():.1f}")

Data coverage statistics:
  Earliest data: 1763
  Latest data: 2026
  Average years of data per station-element: 23.8


In [14]:
# Stations with longest records for core elements
core_elements = ['TMAX', 'TMIN', 'PRCP']
long_records = inventory[
    (inventory['element'].isin(core_elements)) & 
    (inventory['years_of_data'] > 100)
]

print(f"Stations with 100+ years of core data: {long_records['station_id'].nunique():,}")
long_records.sort_values('years_of_data', ascending=False).head(20)

Stations with 100+ years of core data: 6,673


,station_id,latitude,longitude,element,first_year,last_year,years_of_data
139791,ITE00100554,45.4717,9.1892,TMAX,1763,2008,246
139792,ITE00100554,45.4717,9.1892,TMIN,1763,2008,246
129345,GM000010962,47.8017,11.0117,PRCP,1781,2025,245
125620,EZE00100082,50.0864,14.4164,TMIN,1775,2005,231
125619,EZE00100082,50.0864,14.4164,TMAX,1775,2005,231
139796,ITE00105250,38.1103,13.3514,PRCP,1797,2008,212
180256,UK000056225,51.7667,-1.2667,TMAX,1814,2025,212
180257,UK000056225,51.7667,-1.2667,TMIN,1815,2025,211
129329,GM000004204,50.9267,11.5842,TMAX,1824,2025,202
129330,GM000004204,50.9267,11.5842,TMIN,1824,2025,202


## 5. Summary Statistics for Snowflake Loading

Key info to consider when loading into Snowflake:

In [15]:
print("=" * 50)
print("SUMMARY FOR SNOWFLAKE LOADING")
print("=" * 50)
print(f"\nMetadata Tables:")
print(f"  countries:  {len(countries):,} rows")
print(f"  states:     {len(states):,} rows")
print(f"  stations:   {len(stations):,} rows")
print(f"  inventory:  {len(inventory):,} rows")
print(f"\nUnique stations: {stations['station_id'].nunique():,}")
print(f"Unique elements: {inventory['element'].nunique()}")
print(f"Date range: {inventory['first_year'].min()} - {inventory['last_year'].max()}")

SUMMARY FOR SNOWFLAKE LOADING

Metadata Tables:
  countries:  219 rows
  states:     74 rows
  stations:   129,658 rows
  inventory:  767,689 rows

Unique stations: 129,658
Unique elements: 144
Date range: 1763 - 2026
